In [1]:
import glob
import io
import base64

from sklearn.model_selection import train_test_split

import d3rlpy
from d3rlpy.metrics.scorer import evaluate_on_environment

from gym.wrappers     import Monitor

from IPython.display  import HTML
from IPython          import display as ipythondisplay
from pyvirtualdisplay import Display

In [2]:
dataset, env = d3rlpy.datasets.get_atari('breakout-expert-v0')

loading /home/dusodeth/.d4rl/datasets/Breakout/1/50/observation.gz...
loading /home/dusodeth/.d4rl/datasets/Breakout/1/50/action.gz...
loading /home/dusodeth/.d4rl/datasets/Breakout/1/50/reward.gz...
loading /home/dusodeth/.d4rl/datasets/Breakout/1/50/terminal.gz...


- - -

In [4]:
train_episodes, test_episodes = train_test_split(dataset, test_size = 0.1)

In [5]:
cql = d3rlpy.algos.DiscreteCQL(n_frames=4, q_func_factory='qr', scaler='pixel', use_gpu=True)

In [8]:
cql.fit(train_episodes,
        eval_episodes = test_episodes,
        n_epochs = 1,
        scorers  = {
            'environment' : d3rlpy.metrics.scorer.evaluate_on_environment(env),
            'td_error'    : d3rlpy.metrics.scorer.td_error_scorer
        })

2021-06-14 15:21.35 [debug    ] RoundIterator is selected.
2021-06-14 15:21.35 [info     ] Directory is created at d3rlpy_logs/DiscreteCQL_20210614152135
2021-06-14 15:21.35 [debug    ] Fitting scaler...              scaler=pixel
2021-06-14 15:21.35 [info     ] Parameters are saved to d3rlpy_logs/DiscreteCQL_20210614152135/params.json params={'action_scaler': None, 'batch_size': 32, 'encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'gamma': 0.99, 'generated_maxlen': 100000, 'learning_rate': 6.25e-05, 'n_critics': 1, 'n_frames': 4, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'q_func_factory': {'type': 'qr', 'params': {'bootstrap': False, 'share_encoder': False, 'n_quantiles': 32}}, 'real_ratio': 1.0, 'scaler': {'type': 'pixel', 'params': {}}, 'target_reduction_type': 'min', 'target_update_interval': 8000, 'use_gpu': 0, 'algorithm': 'D

Epoch 1/1:   0%|          | 0/27857 [00:00<?, ?it/s]

2021-06-14 15:47.28 [info     ] DiscreteCQL_20210614152135: epoch=1 step=27857 epoch=1 metrics={'time_sample_batch': 0.0002764062066342991, 'time_algorithm_update': 0.007616340615502271, 'loss': 1.8378147763018333, 'time_step': 0.008056897316317242, 'environment': 0.0, 'td_error': 0.1065624305213553} step=27857
2021-06-14 15:47.28 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20210614152135/model_1.pt


[(1,
  {'time_sample_batch': 0.0002764062066342991,
   'time_algorithm_update': 0.007616340615502271,
   'loss': 1.8378147763018333,
   'time_step': 0.008056897316317242,
   'environment': 0.0,
   'td_error': 0.1065624305213553})]

- - -

In [31]:
display = Display(visible=0, size=(1400, 900))
display.start()

In [32]:
def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''
            <video alt="test" autoplay loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
            </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")

In [33]:
env = Monitor(env, './video', force=True)

In [36]:
evaluate_on_environment(env)(cql)

0.0

In [2]:
# show_video()
# d3rlpy record d3rlpy_logs/DiscreteCQL_20210614152135/model_1.pt --env-header 'import gym; from d3rlpy.envs import Atari; env = Atari(gym.make("BreakoutNoFrameskip-v4"), is_eval=True)'

.